## PyTorch实现Logistic Regression

Ref: https://towardsdatascience.com/logistic-regression-on-mnist-with-pytorch-b048327f8d19

In [1]:
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [2]:
train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=False)
test_dataset = dsets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

In [3]:
batch_size = 100
n_iters = 6000
epochs = n_iters / (len(train_dataset) / batch_size)
input_dim = 784
output_dim = 10
lr_rate = 0.001

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

In [5]:
model = LogisticRegression(input_dim, output_dim)
if torch.cuda.is_available():
    model.cuda()

In [6]:
criterion = torch.nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [7]:
iter = 0
for epoch in range(int(epochs)):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.to(device).view(-1, 28 * 28))
        labels = Variable(labels.to(device))

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        iter+=1
        if iter%500==0:
            # calculate Accuracy
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images.to(device).view(-1, 28*28))
                outputs = model(images.to(device))
                _, predicted = torch.max(outputs.data, 1)
                labels = labels.to(device)
                total += labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                correct += (predicted == labels).sum()
            accuracy = 100 * correct/total
            print("Iteration: {}. Loss: {}. Accuracy: {}.".format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 1.8475303649902344. Accuracy: 67.
Iteration: 1000. Loss: 1.5161885023117065. Accuracy: 75.
Iteration: 1500. Loss: 1.432004690170288. Accuracy: 78.
Iteration: 2000. Loss: 1.2119542360305786. Accuracy: 80.
Iteration: 2500. Loss: 1.0666146278381348. Accuracy: 81.
Iteration: 3000. Loss: 0.9026584625244141. Accuracy: 82.
Iteration: 3500. Loss: 1.0693578720092773. Accuracy: 83.
Iteration: 4000. Loss: 0.8095034956932068. Accuracy: 83.
Iteration: 4500. Loss: 0.7517650723457336. Accuracy: 84.
Iteration: 5000. Loss: 0.8090811371803284. Accuracy: 84.
Iteration: 5500. Loss: 0.7685168981552124. Accuracy: 84.
Iteration: 6000. Loss: 0.7125200629234314. Accuracy: 85.
